# Interactive examples with MARS/GRIB input

In [ ]:
%%javascript
// leave this in to disable autoscroll in Jupyter notebook
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Imports

In [ ]:
import os
from pathlib import Path
import json
import fcpy

## Load data

In [ ]:
if (Path().home() / ".ecmwfapirc").exists():
    fpath = "../samples/cases/hplp_ml_q.json"
    request = json.load(open(fpath))["mars"]
    ds = fcpy.fetch_mars_dataset(request)
else:
    ds = fcpy.open_dataset("../data/mars.grib")
    
ds = ds.isel(hybrid=[0, 1])
ds

## Define and run experiment

In [ ]:
suite = fcpy.Suite(
    ds,
    baseline=fcpy.Float(bits=32),
    compressors=[fcpy.LinQuantization(), fcpy.Round()],
    metrics=[fcpy.Difference, fcpy.AbsoluteError],
    bits=list(range(12, 20)),
)

## Lineplot: evaluate the effect of bits

In [ ]:
suite.nblineplot()

## Spatial plots: evaluate the spatial distribution of errors

In [ ]:
if not os.getenv("SPHINX_BUILD"):
    fcpy.nbspatialplot(
        ds,
        baseline=fcpy.Float(bits=32),
        compressor=fcpy.LinQuantization(bits=12),
        third_dim="hybrid",
    )